In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, PolynomialFeatures  # 新增PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline  # 重点使用Pipeline整合流程
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

# 数据加载与预处理（保持原逻辑）
train_data = pd.read_csv('/kaggle/input/moai/Iris_train_data.csv')
test_data = pd.read_csv('/kaggle/input/moai/Iris_test_data.csv')
label_encoder = preprocessing.LabelEncoder()
train_data['Species'] = label_encoder.fit_transform(train_data['Species'])
X_train = train_data.drop(['Species', 'Id'], axis=1)
y_train = train_data.loc[:, 'Species']
X_test = test_data.drop(['Id'], axis=1)

# 构建二阶逻辑回归流水线：多项式特征生成 -> 标准化 -> 逻辑回归
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # 二阶多项式特征（无截距项）
    ('scaler', StandardScaler()),  # 标准化（注意：多项式特征后再标准化更合理）
    ('classifier', LogisticRegression(penalty='l2', C=1.0, max_iter=5000))  # 增加迭代次数防止不收敛
])

# 交叉验证评估（与原逻辑一致，但流程更安全）
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')

print(f"各折准确率: {scores}")
print(f"平均准确率: {scores.mean():.4f}（标准差: {scores.std():.4f}）")

# 如果需要训练最终模型并预测测试集（可选）
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)  # 测试集预测（自动应用多项式+标准化）
res_df = pd.DataFrame({'Id':test_data['Id'], 'Species': label_encoder.inverse_transform(y_pred)})
res_df.to_csv('/kaggle/working/aad.csv', index=False)

各折准确率: [0.91666667 1.         0.95833333 1.         0.875     ]
平均准确率: 0.9500（标准差: 0.0486）
